<a href="https://colab.research.google.com/github/Waadtss/AraMed/blob/main/BERT/AraBERTv2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#bert

In [ ]:
!pip install tensorflow-gpu==1.15.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 411.5 MB 6.2 kB/s 
     |████████████████████████████████| 503 kB 61.8 MB/s 
     |████████████████████████████████| 3.8 MB 50.3 MB/s 
     |████████████████████████████████| 50 kB 5.6 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=bb2a0ba6002d34e9317dbab913f7dec99c52f3438eb2faf1ce9462d832908824
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting u

In [ ]:
import tensorflow as tf

In [ ]:
print(tf.version.VERSION)

1.15.0


In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



#Mount Your Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.cloud import storage

In [ ]:
!curl https://sdk.cloud.google.com | bash

In [ ]:
!gcloud init

#Installing Farasa and pyarabic

To do Farasa segmenting you will need FarasaSegmenter.jar in the same directory as the preprocess.py file 

(you can get the Farasa segmenter from http://qatsdemo.cloudapp.net/farasa/register.html)

In [ ]:
!pip install transformers
# !git clone https://github.com/aub-mind/arabert
!pip install pyarabic
!pip install farasapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.3 MB/s 
     |████████████████████████████████| 120 kB 63.6 MB/s 
     |████████████████████████████████| 6.6 MB 57.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 126 kB 5.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Tensorflow implementation

In [ ]:
#copy from gcp to content
!gsutil --m cp -r gs://BUCKET_NAME/arabert1 /content/

In [ ]:
import tensorflow as tf
from arabert1.preprocess import ArabertPreprocessor
from transformers import AutoTokenizer, AutoModel

model_name = "aubmindlab/bert-base-arabertv2"

In [ ]:
#test BERT tokenizer
bert_tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
text = "الدراسات النظرية للتصاميم الحديثة والمتطورة"
arabert_model = AutoModel.from_pretrained(model_name)
arabert_prep=ArabertPreprocessor(model_name=model_name, keep_emojis=False)

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv2 were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS

100%|██████████| 241M/241M [00:22<00:00, 10.6MiB/s]


[2022-09-03 08:28:30,684 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [ ]:
text_prep = arabert_prep.preprocess(text)
print(text_prep)

ال+ دراس +ات ال+ نظري +ة ل+ ال+ تصاميم ال+ حديث +ة و+ ال+ متطور +ة


In [ ]:
bert_tokenizer.tokenize(text_prep)

['ال+',
 'دراس',
 '+ات',
 'ال+',
 'نظري',
 '+ة',
 'ل+',
 'ال+',
 'تصاميم',
 'ال+',
 'حديث',
 '+ة',
 'و+',
 'ال+',
 'متطور',
 '+ة']

[link text](`https://`)##Tensorflow Training

**ENABLE TPU RUNTIME!!!**

Test Sentiment Analysis score on a dataset like the AJGT

K. M. Alomari, H. M. ElSherif, and K. Shaalan, “Arabic tweets sentimental analysis using machine learning,” in Proceedings of the International Conference on Industrial, Engineering and Other Applications of Applied Intelligent Systems, pp. 602–610, Montreal, Canada, June 2017.

In [ ]:
import os
import sys
import json
import nltk
import random
import logging
import tensorflow as tf
import pandas as pd

from glob import glob
from tensorflow.keras.utils import Progbar
from tqdm import  tqdm

sys.path.append("arabert1")

import arabert1
from arabert1 import arabert 
from arabert1.arabert import modeling, lamb_optimizer, optimization, tokenization
from arabert1.arabert.run_classifier import input_fn_builder, model_fn_builder

from sklearn.model_selection import train_test_split

#You need to authenticate colab when you use TPUs with your models on a google Bucket
from google.colab import auth
auth.authenticate_user()

# configure logging
log = logging.getLogger('tensorflow')
log.setLevel(logging.INFO)

# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s :  %(message)s')
sh = logging.StreamHandler()
sh.setLevel(logging.INFO)
sh.setFormatter(formatter)
log.handlers = [sh]

if 'COLAB_TPU_ADDR' in os.environ:
  log.info("Using TPU runtime")
  USE_TPU = True
  TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']

  with tf.Session(TPU_ADDRESS) as session:
    log.info('TPU address is ' + TPU_ADDRESS)
    # Upload credentials to TPU.
    # with open('./adc.json', 'r') as f:
    #   auth_info = json.load(f)
    # tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
    
else:
  log.warning('Not connected to TPU runtime')
  USE_TPU = False


2022-09-03 08:28:40,698 :  Using TPU runtime
INFO:tensorflow:Using TPU runtime
2022-09-03 08:28:40,785 :  TPU address is grpc://10.23.174.50:8470
INFO:tensorflow:TPU address is grpc://10.23.174.50:8470


In [ ]:
import pandas as pd
import pyarabic.araby as arab
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pyarabic.araby as arab
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import gensim
from gensim.models import KeyedVectors
from gensim.models import word2vec

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
len(x)

103938

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

Train,Test=train_test_split(pd.read_csv("/content/drive/MyDrive/MostRecentQuestions+MostPopularQuestions-automatedAnnotationFix20.csv"),train_size=0.80, random_state=42, shuffle=True)

In [ ]:
Train

In [ ]:
Train

In [ ]:
Train=Train.reset_index(drop=True)
Test=Test.reset_index(drop=True)

In [ ]:
print(len(Train),"\n", len(Test))
print(len(Train)+len(Test))

109834 
 27459
137293


In [ ]:
 Test['Question description']

0        وجهي صغير جدا كيف اكبره ؟ أو هل هناك عمليات تج...
1        سؤالي شو الاكل الممنوع استخدامه بعد الارجاع ال...
2        اريد ان اعرف دواعى استعمال thilo tears ,ومرهم ...
3        انا اجهضت في الشهر الاول من الحمل وجات الدوره ...
4        هل يحتوي المذي على حبونات منوية وهل يسبب الحمل...
                               ...                        
27454    ما امكانيه حدوث حمل اثناء نزول الدورة الشهريه ...
27455    السلام عليكم هل يظهر البنج الموضعي الخارجي في ...
27456                  اريد حمل متزوجة ١٦ سنة ولد واحد فقط
27457    لدي ٢٠ سنة في يوم الدخلة شعرت بألم الدورة ولكن...
27458    ماذا يعني سلس البول انا اشكو من حالة خروج قطرا...
Name: Question description, Length: 27459, dtype: object

In [ ]:
# Train=pd.read_csv("/content/drive/My Drive/Twaseel data/manual-sheet1+sheet3.csv")
# Test=pd.read_csv("/content/drive/My Drive/Twaseel data/CorrectedAnnotatedsheet2+manual.csv")

In [ ]:
def clean_sentence(text):
    #print("before: ", text)
    # if text==['\n'] or ['']:
    #   return 0


      #Strip  tashkeel and tatweel(Elongation) 
      text= arab.strip_tashkeel(text)
      text= arab.strip_tatweel(text)

      text = re.sub("[.]", " ",text)
      text=re.sub(r"https?:\/\/.*[\r\n]*"," ", text, flags = re.MULTILINE)
      text = re.sub(r"[A-Za-z0-9\n\.\،\(\)\{\}\[\] \\ /+ـ_=%٪$#@!:;\-*&÷×\'\ ؛<>\"^`|~]", " ", text, flags=re.UNICODE) #\،\(\)\{\}\[\]+ـ=٪$#@!:;\-
      text=re.sub(r"\s+"," ", text, flags = re.I)

      #normalizeArabic
      text = re.sub("[إأآٱ]", "ا", text)
      text = re.sub("[?]", "؟", text)

      #remove extra spaces
      text = re.sub(' +', ' ', text)
      text = text.split()
      # Astops = set(stopwords.words("arabic"))
      # text = [w for w in text if not w in Astops]

      text = " ".join(text)

    #print("after: ", text)

      return text

In [ ]:
TR_Q=[]
TR_A=[]
TR_label=[]
for n in range(Train.shape[0]):
  TR_Q.append(clean_sentence(Train['Question description'][n]))#+' <CATEGORY> '+s)
  # print((Train['Question description'][n]))
  TR_Q.append(clean_sentence(Train['Question description'][n]))#+' <CATEGORY> '+s)

  TR_A.append(clean_sentence((Train['Answer details 1'][n])))#+' [SEP] '+clean_sentence(Train['الإجابة'][n]))
  TR_label.append(1)
  TR_A.append(clean_sentence(Train['Wrong Answer'][n]))#+' [SEP] '+clean_sentence(Train['الإجابة'][n]))
  TR_label.append(0)
      
TS_Q=[]
TS_A=[]
TS_label=[]
for n in range(Test.shape[0]):
  TS_Q.append(clean_sentence(Test['Question description'][n]))#+' <CATEGORY> '+s)
  TS_Q.append(clean_sentence(Test['Question description'][n]))#+' <CATEGORY> '+s)
  TS_A.append(clean_sentence(Test['Answer details 1'][n]))#+' [SEP] '+clean_sentence(Train['الإجابة'][n]))
  TS_label.append(1)
  TS_A.append(clean_sentence(Test['Wrong Answer'][n]))#+' [SEP] '+clean_sentence(Train['الإجابة'][n]))
  TS_label.append(0)

In [ ]:
print('Train_Q',len(TR_Q),'Train_A',len(TR_A))
print('Train_Qtext',TR_Q[2],'\nTrain_QA',TR_A[2])

print('Test_Q',len(TS_Q),'Test_A',len(TS_A))
print('Test_Q',TS_Q[1],'\nTest_A',TS_A[1])

Train_Q 219668 Train_A 219668
Train_Qtext مارست الجنسه مع خطيبتي يوم بعد دورته الشهريه بدون ايلاج وقذفت على افخاذه وفرجها هل يمكن حدوث حمل 
Train_QA الاحتمال وارد
Test_Q 54918 Test_A 54918
Test_Q وجهي صغير جدا كيف اكبره ؟ او هل هناك عمليات تجميل لتكبير الوجه 
Test_A لابد من فحصه من قبل طبيب تخصص نمائي او اخصائي نفسي و طبيب اطفال


In [ ]:
# !pip install farasapy

In [ ]:
d={"Q":TR_Q,"A":TR_A,"label": TR_label}
train=pd.DataFrame(d)
# train["Qtext"] = train["Qtext"].apply(lambda x: preprocess(x, do_farasa_tokenization=True , farasa=farasa_segmenter, use_farasapy = True))
# train["QA"] = train["QA"].apply(lambda x: preprocess(x, do_farasa_tokenization=True , farasa=farasa_segmenter, use_farasapy = True))


train["Q"] = train["Q"].apply(lambda x: arabert_prep.preprocess(x))
train["A"] = train["A"].apply(lambda x: arabert_prep.preprocess(x))

In [ ]:
dt={"Q":TS_Q ,"A":TS_A, "label":TS_label}
test=pd.DataFrame(dt)
test["Q"] = test["Q"].apply(lambda x: arabert_prep.preprocess(x))
test["A"] = test["A"].apply(lambda x: arabert_prep.preprocess(x))

In [ ]:
test["label"]

0        1
1        0
2        1
3        0
4        1
        ..
54913    0
54914    1
54915    0
54916    1
54917    0
Name: label, Length: 54918, dtype: int64

In [ ]:
train_InputExamples = train.apply(lambda x: arabert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                    text_a = x["Q"], 
                                                                    text_b = x["A"], 
                                                                    label = x["label"]), axis = 1)

test_InputExamples = test.apply(lambda x: arabert.run_classifier.InputExample(guid=None, 
                                                                    text_a = x["Q"], 
                                                                    text_b = x["A"], 
                                                                    label = x["label"]), axis = 1)

In [ ]:
labels = list(train.label.unique())
print(labels)

[1, 0]


In [ ]:
labels = list(train.label.unique())
print("labels",labels)
MAX_SEQ_LENGTH = 256
train_features = arabert.run_classifier.convert_examples_to_features(train_InputExamples, labels, MAX_SEQ_LENGTH, bert_tokenizer)
test_features = arabert.run_classifier.convert_examples_to_features(test_InputExamples, labels, MAX_SEQ_LENGTH, bert_tokenizer)

2022-09-03 08:54:13,263 :  From /content/arabert1/arabert/run_classifier.py:853: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.


2022-09-03 08:54:13,265 :  Writing example 0 of 219668
INFO:tensorflow:Writing example 0 of 219668
2022-09-03 08:54:13,270 :  *** Example ***
INFO:tensorflow:*** Example ***
2022-09-03 08:54:13,272 :  guid: None
INFO:tensorflow:guid: None
2022-09-03 08:54:13,275 :  tokens: [CLS] هل ضغط ال+ دم مرتفع ؟ [SEP] ان ضغط ال+ دم يتكون من رقم +ين و+ +هما ال+ رقم ال+ علوي و+ ال+ رقم ال+ سفلي و+ يمكن توضيح +هم كما يلي ال+ رقم ال+ علوي الذي يشير الى ضغط ال+ دم ال+ انقباضي و+ الذي يجب ان يكون ب+ صور +ة طبيعي +ة بين مم زئبق +ي ال+ رقم ال+ سفلي الذي يشير الى ضغط ال+ دم ال+ انبساط ##ي الذي يجب ان يكون ب+ صور +ة طبيعي +ة بين مم زئبق +ي ان ضغط ال+ دم ال+ انقباضي الذي يكون أعلى من مم زئبق +ي يعتبر ارتفاع في ضغط ال+ دم و+ ب+ ال+ تالي فان إجاب +ة سؤال هل ضغط ال+ دم مرتفع هي نعم و+ يجب ال+ علم ان مرض ارتفاع ضغط ال+ دم يعرف ايض +ا باسم ال+ قات

labels [1, 0]


2022-09-03 08:54:19,405 :  Writing example 10000 of 219668
INFO:tensorflow:Writing example 10000 of 219668
2022-09-03 08:54:25,495 :  Writing example 20000 of 219668
INFO:tensorflow:Writing example 20000 of 219668
2022-09-03 08:54:31,956 :  Writing example 30000 of 219668
INFO:tensorflow:Writing example 30000 of 219668
2022-09-03 08:54:38,013 :  Writing example 40000 of 219668
INFO:tensorflow:Writing example 40000 of 219668
2022-09-03 08:54:44,004 :  Writing example 50000 of 219668
INFO:tensorflow:Writing example 50000 of 219668
2022-09-03 08:54:50,025 :  Writing example 60000 of 219668
INFO:tensorflow:Writing example 60000 of 219668
2022-09-03 08:54:56,794 :  Writing example 70000 of 219668
INFO:tensorflow:Writing example 70000 of 219668
2022-09-03 08:55:02,955 :  Writing example 80000 of 219668
INFO:tensorflow:Writing example 80000 of 219668
2022-09-03 08:55:09,057 :  Writing example 90000 of 219668
INFO:tensorflow:Writing example 90000 of 219668
2022-09-03 08:55:15,146 :  Writing ex

#Arabert checkpint and config

In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash

Detected operating system as Ubuntu/bionic.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... done.
Running apt-get update... done.

The repository is setup! You can now install packages.


In [ ]:
!sudo apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following packages will be upgraded:
  git-lfs
1 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
Need to get 7,168 kB of archives.
After this operation, 7,962 kB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu bionic/main amd64 git-lfs amd64 3.2.0 [7,168 kB]
Fetched 7,168 kB in 0s (14.8 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-

In [ ]:
!git lfs install

Git LFS initialized.


In [ ]:
!git clone https://huggingface.co/aubmindlab/bert-base-arabertv2/

Cloning into 'bert-base-arabertv2'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 48 (delta 12), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (48/48), done.
Filtering content: 100% (4/4), 2.17 GiB | 119.27 MiB/s, done.


In [ ]:
!tar -C ./bert-base-arabertv2 -zxvf /content/bert-base-arabertv2/tf1_model.tar.gz

tf-base-arabertv2/
tf-base-arabertv2/config.json
tf-base-arabertv2/model.ckpt.data-00000-of-00001
tf-base-arabertv2/vocab.txt
tf-base-arabertv2/model.ckpt.index
tf-base-arabertv2/checkpoint
tf-base-arabertv2/model.ckpt.meta


In [ ]:
ls bert-base-arabertv2

config.json         special_tokens_map.json  tokenizer_config.json
flax_model.msgpack  tf1_model.tar.gz         tokenizer.json
pytorch_model.bin   tf-base-arabertv2/       vocab.txt
README.md           tf_model.h5


In [ ]:
!cp -av '/content/bert-base-arabertv2' '/content/drive/MyDrive/bert-base-arabertv2'

cp: cannot create directory '/content/drive/MyDrive/bert-base-arabertv2': No such file or directory


In [ ]:
!gsutil --m cp -r /content/bert-base-arabertv2 gs://sawas/

Copying file:///content/bert-base-arabertv2/special_tokens_map.json [Content-Type=application/json]...
Copying file:///content/bert-base-arabertv2/tf1_model.tar.gz [Content-Type=application/x-tar]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Copying file:///content/bert-base-arabertv2/tokenizer_config.json [Content-Type=application/json]...
Copying file:///content

In [ ]:
# BERT_PRETRAINED_DIR = '/content/bert-base-arabertv02/tf-base-arabertv02/'

#AraBEERT finetuinig 

In [ ]:
BERT_PRETRAINED_DIR = 'gs://sawas/bert-base-arabertv2/tf-base-arabertv2/'

In [ ]:
# Input data pipeline config
TRAIN_BATCH_SIZE = 16 #@param {type:"integer"} #You can probably 
                                              #increase when using TPUS
MAX_SEQ_LENGTH = 256 #@param {type:"integer"} #reduce if running on GPU

# Training procedure config
EVAL_BATCH_SIZE = 16 
LEARNING_RATE = 2e-5 #between 1e-5 and 6e-5
NUM_TRAIN_EPOCHS = 6 #@param {type:"integer"}
WARMUP_PROPORTION = 0.1 #@param {type:"number"}
NUM_TPU_CORES = 8

PREDICT_BATCH_SIZE = 8


CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, "config.json")

#change the model name when you use arabertv0.1
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR,"model.ckpt")
OUTPUT_DIR_PER_MODEL = 'gs://sawas/output/bert-base-arabertv2/new/1/'
bert_config = modeling.BertConfig.from_json_file(CONFIG_FILE)

log.info("Using checkpoint: {}".format(INIT_CHECKPOINT))

print("CONFIG_FILE: "+CONFIG_FILE)
print("INIT_CHECKPOINT: "+INIT_CHECKPOINT)
print("OUTPUT_DIR_PER_MODEL: "+OUTPUT_DIR_PER_MODEL)

2022-09-03 09:00:42,299 :  Using checkpoint: gs://sawas/bert-base-arabertv2/tf-base-arabertv2/model.ckpt
INFO:tensorflow:Using checkpoint: gs://sawas/bert-base-arabertv2/tf-base-arabertv2/model.ckpt


CONFIG_FILE: gs://sawas/bert-base-arabertv2/tf-base-arabertv2/config.json
INIT_CHECKPOINT: gs://sawas/bert-base-arabertv2/tf-base-arabertv2/model.ckpt
OUTPUT_DIR_PER_MODEL: gs://sawas/output/bert-base-arabertv2/new/1/


In [ ]:
num_train_steps = int(len(train_features) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)
num_steps_per_epoch = int(len(train_features) / TRAIN_BATCH_SIZE)

print("num train steps: {}".format(num_train_steps))
print("num warmup steps: {}".format(num_warmup_steps))
print("num_steps_per_epoch: {}".format(num_steps_per_epoch))

model_fn = model_fn_builder(
  bert_config=modeling.BertConfig.from_json_file(CONFIG_FILE),
  num_labels=2,
  init_checkpoint=INIT_CHECKPOINT,
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=USE_TPU,
  use_one_hot_embeddings=USE_TPU
)

tpu_cluster_resolver = None
if USE_TPU:
  tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)

run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=OUTPUT_DIR_PER_MODEL,
    save_checkpoints_steps=num_steps_per_epoch,
    keep_checkpoint_max=0,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=num_steps_per_epoch,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=USE_TPU,
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE,
    predict_batch_size=PREDICT_BATCH_SIZE)
  
train_input_fn = input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)

# Dev_input_fn = input_fn_builder(
#     features=Dev_features,
#     seq_length=MAX_SEQ_LENGTH,
#     is_training=False,
#     drop_remainder=True)

test_input_fn = input_fn_builder(
  features=test_features,
  seq_length=MAX_SEQ_LENGTH,
  is_training=False,
  drop_remainder=True)

2022-09-03 09:00:50,688 :  
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

2022-09-03 09:00:50,695 :  Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7fc687acaef0>) includes params argument, but params are not passed to Estimator.
2022-09-03 09:00:50,699 :  Using config: {'_model_dir': 'gs://sa

num train steps: 82375
num warmup steps: 8237
num_steps_per_epoch: 13729


##Train the model

### train

In [ ]:
import time, datetime
# starting time
start = datetime.datetime.now()
# program body starts
#
#


print(f'Beginning Training!')
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)


#
#
# program body ends
time.sleep(1)
# end time
end = datetime.datetime.now()
diff = (end - start) 
datetime.timedelta(seconds=10, microseconds=885206)
# total time taken
print(diff)
diff_seconds = int(diff.total_seconds())
minute_seconds, seconds = divmod(diff_seconds, 60)
hours, minutes = divmod(minute_seconds, 60)
# total time taken in h m s
hms = f"{hours}h {minutes}m {seconds}s"
print(hms)

2022-09-03 09:00:55,367 :  Querying Tensorflow master (grpc://10.23.174.50:8470) for TPU system metadata.
INFO:tensorflow:Querying Tensorflow master (grpc://10.23.174.50:8470) for TPU system metadata.
2022-09-03 09:00:55,379 :  Found TPU system:
INFO:tensorflow:Found TPU system:
2022-09-03 09:00:55,382 :  *** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Cores: 8
2022-09-03 09:00:55,384 :  *** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Workers: 1
2022-09-03 09:00:55,387 :  *** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
2022-09-03 09:00:55,389 :  *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 3534167116142149908)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 3534167116142149908)
2022-09-03 09:00:55,392 :  *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 6105862917940575187)
INFO:tensorflow:*** A

Beginning Training!


2022-09-03 09:00:55,479 :  From /usr/local/lib/python3.7/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
2022-09-03 09:00:55,482 :  From /usr/local/lib/python3.7/dist-packages/tensorflow_core/python/training/training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
2

1:09:40.162091
1h 9m 40s


##Evaluate the model on all saved checkpoint files

In [ ]:
import time, datetime
# starting time
start = datetime.datetime.now()
# program body starts
#
#


print(f'Beginning Evaluation!')
eval_model_files = tf.gfile.Glob(os.path.join(OUTPUT_DIR_PER_MODEL,'*index'))

for eval_checkpoint in tqdm(sorted(eval_model_files,key=lambda x: int(x[0:-6].split('-')[-1]))):
  result = estimator.evaluate(input_fn=test_input_fn, steps=int(len(test_features)/EVAL_BATCH_SIZE),checkpoint_path=eval_checkpoint[0:-6])
  tf.logging.info("***** Test Eval results *****")
  for key in sorted(result.keys()):
    tf.logging.info("  %s = %s", key, str(result[key]))


#
#
# program body ends
time.sleep(1)
# end time
end = datetime.datetime.now()
diff = (end - start) 
datetime.timedelta(seconds=10, microseconds=885206)
# total time taken
print(diff)
diff_seconds = int(diff.total_seconds())
minute_seconds, seconds = divmod(diff_seconds, 60)
hours, minutes = divmod(minute_seconds, 60)
# total time taken in h m s
hms = f"{hours}h {minutes}m {seconds}s"
print(hms)

Beginning Evaluation!


  0%|          | 0/8 [00:00<?, ?it/s]2022-09-03 10:10:35,507 :  Calling model_fn.
INFO:tensorflow:Calling model_fn.
2022-09-03 10:11:26,239 :  *** Features ***
INFO:tensorflow:*** Features ***
2022-09-03 10:11:26,241 :    name = input_ids, shape = (2, 256)
INFO:tensorflow:  name = input_ids, shape = (2, 256)
2022-09-03 10:11:26,245 :    name = input_mask, shape = (2, 256)
INFO:tensorflow:  name = input_mask, shape = (2, 256)
2022-09-03 10:11:26,247 :    name = label_ids, shape = (2,)
INFO:tensorflow:  name = label_ids, shape = (2,)
2022-09-03 10:11:26,250 :    name = segment_ids, shape = (2, 256)
INFO:tensorflow:  name = segment_ids, shape = (2, 256)
2022-09-03 10:11:29,213 :  **** Trainable Variables ****
INFO:tensorflow:**** Trainable Variables ****
2022-09-03 10:11:29,216 :    name = bert/embeddings/word_embeddings:0, shape = (64000, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (64000, 768), *INIT_FROM_CKPT*
2022-09-03 10:11:29,218 :    

0:21:48.242957
0h 21m 48s
